# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def nhl_correlation(): 
    # YOUR CODE HERE
    global nhl_df
    global cities
    
    nhl_df_cpoy = nhl_df.copy()
    cities_copy = cities.copy()
    
    cities_copy['NHL'] = cities_copy['NHL'].str.split('[').str[0]
    cities_copy = cities_copy[cities_copy['NHL'].apply(len) >1]

    area_team_dict = dict(zip(cities_copy['NHL'],cities_copy['Metropolitan area']))
    area_team_dict['Rangers'] = 'New York City'
    area_team_dict['Islanders'] = 'New York City'
    area_team_dict['Devils'] = 'New York City'
    area_team_dict['Kings'] = 'Los Angeles'
    area_team_dict['Ducks'] = 'Los Angeles'
    area_team_dict

    nhl_df_cpoy = nhl_df_cpoy[nhl_df_cpoy['year'] == 2018]
    nhl_df_cpoy = nhl_df_cpoy[~nhl_df_cpoy['team'].str.contains('Division', na=False)]
    nhl_df_cpoy['team'] = nhl_df_cpoy['team'].str.split('*').str[0]

    def find_team_area(team):
        for key, val in area_team_dict.items():
            if key in team:
                return val

    nhl_df_cpoy['Metropolitan area'] = nhl_df_cpoy['team'].apply(find_team_area)
    
    nhl_df_cpoy['W'] = nhl_df_cpoy['W'].astype(float)
    nhl_df_cpoy['L'] = nhl_df_cpoy['L'].astype(float)
    
    nhl_df_cpoy = nhl_df_cpoy.groupby('Metropolitan area').agg({'W':sum, 'L':sum})
    nhl_df_cpoy['WL'] = nhl_df_cpoy['W'] / (nhl_df_cpoy['L'] + nhl_df_cpoy['W'])
    
    
    
    merged_df = cities_copy.merge(nhl_df_cpoy, left_on='Metropolitan area', right_on = 'Metropolitan area')
            
    population_by_region = merged_df['Population (2016 est.)[8]'].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['WL'].astype(float) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

#     population_by_region = [merged_df['Population (2016 est.)[8]']] # pass in metropolitan area population from cities
#     win_loss_by_region = [merged_df['WL']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

#     assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
#     print(stats.pearsonr(population_by_region, win_loss_by_region)[0])
#     return merged_df
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.012308996455744289


,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL,W,L,WL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils,113.0,105.0,0.518349
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,89.0,54.0,0.622378
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks,45.0,27.0,0.625000
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks,33.0,39.0,0.458333
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars,42.0,32.0,0.567568
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals,49.0,26.0,0.653333
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers,42.0,26.0,0.617647
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins,50.0,20.0,0.714286
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild,45.0,26.0,0.633803
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche,43.0,30.0,0.589041


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [31]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    global nba_df
    global cities
    
    nba_df_cpoy = nba_df.copy()
    cities_copy = cities.copy()
    
    cities_copy['NBA'] = cities_copy['NBA'].str.split('[').str[0]
    cities_copy = cities_copy[cities_copy['NBA'].apply(len) >1]

    area_team_dict = dict(zip(cities_copy['NBA'],cities_copy['Metropolitan area']))
    area_team_dict['Knicks'] = 'New York City'
    area_team_dict['Nets'] = 'New York City'
    area_team_dict['Lakers'] = 'Los Angeles'
    area_team_dict['Clippers'] = 'Los Angeles'
    area_team_dict
    
    nba_df_cpoy = nba_df_cpoy[nba_df_cpoy['year'] == 2018]
    nba_df_cpoy['team'] = nba_df_cpoy['team'].str.split('*').str[0]

    def find_team_area(team):
        for key, val in area_team_dict.items():
            if key in team:
                return val

    nba_df_cpoy['Metropolitan area'] = nba_df_cpoy['team'].apply(find_team_area)
    nba_df_cpoy = nba_df_cpoy[['Metropolitan area','W/L%']]
    nba_df_cpoy['W/L%'] = nba_df_cpoy['W/L%'].astype(float)
    nba_df_cpoy = nba_df_cpoy.groupby('Metropolitan area').mean()
    
    merged_df = cities_copy.merge(nba_df_cpoy, left_on='Metropolitan area', right_on = 'Metropolitan area')
            
    population_by_region = merged_df['Population (2016 est.)[8]'].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['W/L%'].astype(float) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    
#     print(cities_copy.shape)
#     return merged_df
    
#     population_by_region = [] # pass in metropolitan area population from cities
#     win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [40]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    global nba_df
    global cities
    
    mlb_df_cpoy = mlb_df.copy()
    cities_copy = cities.copy()
    
    cities_copy['MLB'] = cities_copy['MLB'].str.split('[').str[0]
    cities_copy = cities_copy[cities_copy['MLB'].apply(len) >1]

    area_team_dict = dict(zip(cities_copy['MLB'],cities_copy['Metropolitan area']))
    area_team_dict['Yankees'] = 'New York City'
    area_team_dict['Mets'] = 'New York City'
    area_team_dict['Dodgers'] = 'Los Angeles'
    area_team_dict['Angels'] = 'Los Angeles'
    area_team_dict['Giants'] = 'San Francisco Bay Area'
    area_team_dict['Athletics'] = 'San Francisco Bay Area'
    area_team_dict['Cubs'] = 'Chicago'
    area_team_dict['White Sox'] = 'Chicago'
    area_team_dict
    
    mlb_df_cpoy = mlb_df_cpoy[mlb_df_cpoy['year'] == 2018]
    mlb_df_cpoy['team'] = mlb_df_cpoy['team'].str.split('*').str[0]

    def find_team_area(team):
        for key, val in area_team_dict.items():
            if key in team:
                return val

    mlb_df_cpoy['Metropolitan area'] = mlb_df_cpoy['team'].apply(find_team_area)
    mlb_df_cpoy = mlb_df_cpoy[['Metropolitan area','W-L%']]
    mlb_df_cpoy['W-L%'] = mlb_df_cpoy['W-L%'].astype(float)
    mlb_df_cpoy = mlb_df_cpoy.groupby('Metropolitan area').mean()
    
    merged_df = cities_copy.merge(mlb_df_cpoy, left_on='Metropolitan area', right_on = 'Metropolitan area')
            
    population_by_region = merged_df['Population (2016 est.)[8]'].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['W-L%'].astype(float) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]


    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


mlb_correlation()

(26, 1)


0.15003737475409493

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [55]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    
    nfl_df_cpoy = nfl_df.copy()
    cities_copy = cities.copy()
    
    cities_copy['NFL'] = cities_copy['NFL'].str.split('[').str[0]
    cities_copy = cities_copy[cities_copy['NFL'].apply(len) >1]

    area_team_dict = dict(zip(cities_copy['NFL'],cities_copy['Metropolitan area']))
    area_team_dict['Giants'] = 'New York City'
    area_team_dict['Jets'] = 'New York City'
    area_team_dict['Rams'] = 'Los Angeles'
    area_team_dict['Chargers'] = 'Los Angeles'
    area_team_dict['49ers'] = 'San Francisco Bay Area'
    area_team_dict['Raiders'] = 'San Francisco Bay Area'
    area_team_dict
    
    nfl_df_cpoy = nfl_df_cpoy[nfl_df_cpoy['year'] == 2018]
    nfl_df_cpoy['team'] = nfl_df_cpoy['team'].str.split('*').str[0]

    def find_team_area(team):
        for key, val in area_team_dict.items():
            if key in team:
                return val

    nfl_df_cpoy['Metropolitan area'] = nfl_df_cpoy['team'].apply(find_team_area)
    nfl_df_cpoy = nfl_df_cpoy[['Metropolitan area','W-L%']]
    nfl_df_cpoy = nfl_df_cpoy[~nfl_df_cpoy['Metropolitan area'].isnull()]
    nfl_df_cpoy['W-L%'] = nfl_df_cpoy['W-L%'].astype(float)
    nfl_df_cpoy = nfl_df_cpoy.groupby('Metropolitan area').mean()
    
    merged_df = cities_copy.merge(nfl_df_cpoy, left_on='Metropolitan area', right_on = 'Metropolitan area')
            
    population_by_region = merged_df['Population (2016 est.)[8]'].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['W-L%'].astype(float) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]


    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nfl_correlation()

0.004282141436393019

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values